# Biblioteki

In [92]:
import json
import os
import requests
from bs4 import BeautifulSoup

# Zadanie 1

## Wysłanie żądania dostępu do zasobu serwera 

In [93]:
product_id = "114700014"
url = f"https://www.ceneo.pl/{product_id}/opinie-1"

# https://www.ceneo.pl/114700014/opinie-1

In [94]:
all_opinions = []

while (url):
    response = requests.get(url)
    page_dom = BeautifulSoup(response.text, "html.parser")
    #print(type(page_dom))

    opinions = page_dom.select("div.js_product-review")
    #print(type(opinions))

    for opinion in opinions:
        try:
            single_opinion = {
                "opinion_id"      : opinion["data-entry-id"],
                "author"          : opinion.select_one("span.user-post__author-name").get_text().strip(),
                "recommendation"  : opinion.select_one("span.user-post__author-recomendation > em").get_text().strip(),
                "rating"          : opinion.select_one("span.user-post__score-count").get_text().strip(),
                "content"         : opinion.select_one("div.user-post__text").get_text().strip(),
                "pros"            : [p.get_text().strip() for p in opinion.select(".review-feature__title--positives~.review-feature__item")],
                "cons"            : [c.get_text().strip() for c in opinion.select(".review-feature__title--negatives~.review-feature__item")],
                "useful"          : opinion.select_one('span[id^="votes-yes"]').get_text().strip(),
                "useless"         : opinion.select_one('span[id^="votes-no"]').get_text().strip(),
                "post_date"       : opinion.select_one(".user-post__published > time:nth-child(1)")["datetime"].strip(),
                "purchase_date"   : opinion.select_one(".user-post__published > time:nth-child(2)")["datetime"].strip()
            }
        
            all_opinions.append(single_opinion)

        except (AttributeError, TypeError):
            pass
    try: 
        url = "https://www.ceneo.pl" + page_dom.select_one("div.pagination__next")["href"]
    except TypeError:
        url = False

## Ekstrakcja składowych pojedynczej opinii 

|Składowa|Selektor|Zmienna|
|--------|--------|-------|
|id opinii|["data-entry-id"]|opinion_id|
|autor|span.user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation>em|recommendation|
|gwiazdki|span.user-post__score-count|rating|
|treść|div.user-post__text|content|
|lista zalet|.review-feature__title--positives~.review-feature__item|pros|
|lista wad|.review-feature__title--negatives~.review-feature__item|cons|
|dla ilu przydatna|[id^="votes-yes"]|useful|
|dla ilu nieprzydatna|[id^="votes-no"]|useless|
|data wystawienia|.user-post__published > time:nth-child(1)["datetime"]|post_date|
|data zakupu|.user-post__published > time:nth-child(2)["datetime"]|purchase_date|

In [95]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")

with open(f"opinions/{product_id}.json", "w", encoding="UTF-8") as jfile:
    json.dump(all_opinions, jfile, indent=6, ensure_ascii=False)